In [1]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
from hplib import hplib as hpl
import plotly.graph_objects as go
from PLZtoWeatherRegion import getregion

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)#, url_base_pathname='/pvsym22/')

df = pd.read_pickle('results_summary_new.pkl')
heatpumps=hpl.load_database()
heatpumps=heatpumps[['Manufacturer', 'Model', 'Date', 'SPL indoor [dBA]', 'SPL outdoor [dBA]', 'PSB [W]', 'P_th_h_ref [W]','MAPE_P_el', 'MAPE_COP', 'MAPE_P_th',
       'P_th_c_ref [W]', 'MAPE_P_el_cooling', 'MAPE_EER', 'MAPE_Pdc']]
region=['Nordseeküste','Ostseeküste','Nordwestdeutsches Tiefland','Nordostdeutsches Tiefland','Niederrheinisch-westfälische Bucht und Emsland','Nördliche und westliche Mittelgebirge, Randgebiete',
'Nördliche und westliche Mittelgebirge, zentrale Bereiche','Oberharz und Schwarzwald (mittlere Lagen)','Thüringer Becken und Sächsisches Hügelland','Südöstliche Mittelgebirge bis 1000 m',
'Erzgebirge, Böhmer- und Schwarzwald oberhalb 1000 m','Oberrheingraben und unteres Neckartal','Schwäbisch-fränkisches Stufenland und Alpenvorland','Schwäbische Alb und Baar',
'Alpenrand und -täler']
app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Input(
                placeholder='Wohnort oder PLZ',
                type='text',
                value='',
                id='Standort',            
            ),'enspricht Wetterregion: ',dcc.Dropdown(region,
                        id='region'),
            html.Div("Gebäudetyp: "),
            dcc.Dropdown(
                df['Gebäudetyp'].unique(),
                'Neubau (150 Qm und 0.6 W/(K*Qm))',
                id='sort2',
            ),
            "PV-Ausrichtung: ",
            dcc.Dropdown(
                df['PV-Ausrichtung'].unique(),
                'Süd',
                id='sort3',
            ),
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            "Strombezugskosten in ct/kWh: ",
            dcc.Slider(28, 40, 1,
               value=35,
               id='strombezugskosten'
            ),
            "Einspeisevergütung in ct/kWh: ",
            dcc.Slider(0, 12, 1,
               value=6,
               id='einspeisevergütung'
            ),
            
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),
    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'curveNumber': 0,'x':'geregelt','hovertext': 'Generic Luft/Wasser geregelt'}]},
            clickData={'points': [{'curveNumber': 0,'x':'geregelt','hovertext': 'Generic Luft/Wasser geregelt'}]}
        )
    ], style={'width': '100%', 'display': 'inline-block', 'padding': '10 50'}),
    html.Div([
        dcc.Graph(
            id='graph2',
            hoverData={'points': [{'curveNumber': 0,'x':'geregelt','hovertext': 'Generic Luft/Wasser geregelt'}]},
            clickData={'points': [{'curveNumber': 0,'x':'geregelt','hovertext': 'Generic Luft/Wasser geregelt'}]}
        )
    ]),
    html.Div([
        dcc.Markdown(
         id='wp-infos')
    ]),
])


@app.callback(
    Output('region', 'value'),
    Input('Standort', 'value'))
def standorttoregion(standort):
    return region[getregion(standort)-1]

@app.callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('region', 'value'),
    Input('sort2', 'value'),
    Input('sort3', 'value'),
    Input('strombezugskosten', 'value'),
    Input('einspeisevergütung', 'value'),
    )
def update_graph(standort, gebäudetyp,pv,strombezugskosten, einspeisevergütung):
    dff = df[(df['Standort'] == region.index(standort)+1)&(df['Gebäudetyp']==gebäudetyp)&(df['Jahr']==2015)&(df['Typ']=='durchschnittliches Jahr')&(df['Batteriespeicher [kWh]']==0)&(df['PV-Ausrichtung']==pv)]
    dff['bilanzielle Energiekosten'] = dff['Netzbezug [kWh]'].values * strombezugskosten/100 - dff['Netzeinspeisung [kWh]'].values * einspeisevergütung/100
    dff.loc[dff['WP-Name']=='Generic','WP-Name']='Generic '+ dff.loc[dff['WP-Name']=='Generic','WP-Kategorie'] +' '+ dff.loc[dff['WP-Name']=='Generic','WP-Typ']
    
    dff=dff.sort_values('bilanzielle Energiekosten')
    fig=px.bar(data_frame=dff,
                    y='WP-Name',                    
                    x='bilanzielle Energiekosten',
                    hover_name='WP-Name',
                    hover_data=['WP-Hersteller'],
                    color='WP-Kategorie',
                    #title='Strombezug abzüglich Einspeisevergütung eines EFH mit 10 kWp, 0 kWh Batteriespeicher',
                    labels=dict(x='Bilanzielle Stromkosten [€/a]',y='WP-Models',color='WP-Kategorie'),
                    height=600
            ).update_yaxes(categoryorder='total descending')
    fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.001,
    xanchor="left"
    ))
    fig.update_xaxes(range=[dff['bilanzielle Energiekosten'].min()*0.9,dff['bilanzielle Energiekosten'].max()*1.05])
    fig.update_layout(xaxis_title='Bilanzielle Stromkosten [€/a]',
                yaxis_title='WP-Model')
    if fig['data'][0]['legendgroup']=='Sole/Wasser':
        fig['data'][0]['marker']['color']='#636efa'
        fig['data'][1]['marker']['color']='#EF553B'
    else:
        fig['data'][1]['marker']['color']='#636efa'
        fig['data'][0]['marker']['color']='#EF553B'
    return fig

@app.callback(
    Output('graph2', 'figure'),
    Input('crossfilter-indicator-scatter', 'clickData'),
    Input('region', 'value'),
    Input('sort2', 'value'),
    Input('sort3', 'value'),
    Input('strombezugskosten', 'value'),
    Input('einspeisevergütung', 'value'),
    )
def update_graph2(wp_name,standort, gebäudetyp, pv, strombezugskosten, einspeisevergütung):
    wpname=wp_name['points'][0]['hovertext']

    if wp_name['points'][0]['hovertext'].startswith('Generic'):
        if wp_name['points'][0]['hovertext'].endswith('Luft/Wasser einstufig'):
            dff = df[(df['Standort'] == region.index(standort)+1)&(df['Gebäudetyp']==gebäudetyp)&(df['PV-Ausrichtung']==pv)&(df['WP-Name']=='Generic')&(df['WP-Kategorie']=='Luft/Wasser')&(df['WP-Typ']=='einstufig')]
        elif wp_name['points'][0]['hovertext'].endswith('Luft/Wasser geregelt'):
            dff = df[(df['Standort'] == region.index(standort)+1)&(df['Gebäudetyp']==gebäudetyp)&(df['PV-Ausrichtung']==pv)&(df['WP-Name']=='Generic')&(df['WP-Kategorie']=='Luft/Wasser')&(df['WP-Typ']=='geregelt')]
        elif wp_name['points'][0]['hovertext'].endswith('einstufig'):
            dff = df[(df['Standort'] == region.index(standort)+1)&(df['Gebäudetyp']==gebäudetyp)&(df['PV-Ausrichtung']==pv)&(df['WP-Name']=='Generic')&(df['WP-Kategorie']!='Luft/Wasser')&(df['WP-Typ']=='einstufig')]
        elif wp_name['points'][0]['hovertext'].endswith('geregelt'):
            dff = df[(df['Standort'] == region.index(standort)+1)&(df['Gebäudetyp']==gebäudetyp)&(df['PV-Ausrichtung']==pv)&(df['WP-Name']=='Generic')&(df['WP-Kategorie']!='Luft/Wasser')&(df['WP-Typ']=='geregelt')]
    else:
        dff = df[(df['Standort'] == region.index(standort)+1)&(df['Gebäudetyp']==gebäudetyp)&(df['PV-Ausrichtung']==pv)&(df['WP-Name']==wpname)]
    
    dff['Kosten [1/Jahr]'] = dff['Netzbezug [kWh]'].values * strombezugskosten/100 - dff['Netzeinspeisung [kWh]'].values * einspeisevergütung/100

    fig = px.box(dff, x='Batteriespeicher [kWh]',y='Kosten [1/Jahr]', points='all', hover_data=['Typ', 'Jahr'])
    fig.update_layout(title='Bilanzielle Stromkosten mit '+wpname+' Wärmepumpe über Batteriegröße in verschiedenen Wetterjahren',
                xaxis_title='Batteriespeicher [kWh]',
                yaxis_title='Bilanzielle Stromkosten [€/a]')
    return fig

@app.callback(
    Output('wp-infos', 'children'),
    Input('crossfilter-indicator-scatter', 'clickData'),
    Input('graph2', 'clickData'))
def update_table(wp_name, Wp_name):
    wpname=wp_name['points'][0]['hovertext']
    hp=heatpumps.loc[heatpumps['Model']==wpname]
    try:
        samehp=''
        for i in hpl.same_built_type(wpname):
            samehp=samehp+i+' \n'
        samehp=samehp[:-2]
        samehp
        hp['Modelnamen']=samehp
        hp=hp[['Manufacturer','Modelnamen', 'SPL indoor [dBA]', 'PSB [W]',
            'MAPE_COP']].rename(columns={'Manufacturer':'Hersteller','SPL indoor [dBA]':'Geräuschpegel in dBA','PSB [W]': 'Standby Leistung in W'})
    except:
        pass
    return hp.to_markdown()

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [30/May/2022 17:15:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 17:15:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 17:15:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 17:15:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [30/May/2022 17:15:54] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [30/May/2022 17:15:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [30/May/2022 17:15:54] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [30/May/2022 17:15:54] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [30/May/2022 17:15:55] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [30/May/2022 17:15:55] "POST /_dash-update-component HTTP/1.1" 200 

In [15]:
strombezugskosten = 31
einspeisevergütung = 6
batcost = 100
pvcost = 12
standort = 4
gebäudetyp = 'Neubau (150 Qm und 0.6 W/(K*Qm))'
pv = 'Süd'
wpname='HA 3-6 O 230V'

In [19]:
dff = df[(df['Standort'] == 4)&(df['Gebäudetyp']==gebäudetyp)&(df['Jahr']==2015)&(df['Typ']=='durchschnittliches Jahr')&(df['Batteriespeicher [kWh]']==0)&(df['PV-Ausrichtung']==pv)]
dff['bilanzielle Energiekosten'] = dff['Netzbezug [kWh]'].values * strombezugskosten/100 - dff['Netzeinspeisung [kWh]'].values * einspeisevergütung/100
dff.loc[dff['WP-Name']=='Generic','WP-Name']='Generic '+ dff.loc[dff['WP-Name']=='Generic','WP-Kategorie'] +' '+ dff.loc[dff['WP-Name']=='Generic','WP-Typ']

dff=dff.sort_values('bilanzielle Energiekosten')
fig=px.bar(data_frame=dff,
                y='WP-Name',                    
                x='bilanzielle Energiekosten',
                hover_name='WP-Name',
                hover_data=['WP-Hersteller'],
                color='WP-Kategorie',
                #title='Strombezug abzüglich Einspeisevergütung eines EFH mit 10 kWp, 0 kWh Batteriespeicher',
                labels=dict(x='Bilanzielle Stromkosten [€/a]',y='WP-Models',color='WP-Kategorie'),
                height=600
        ).update_yaxes(categoryorder='total descending')
fig.update_layout(legend=dict(
orientation="h",
yanchor="bottom",
y=1.001,
xanchor="left"
))
fig.update_xaxes(range=[dff['bilanzielle Energiekosten'].min()*0.9,dff['bilanzielle Energiekosten'].max()*1.05])
fig.update_layout(xaxis_title='Bilanzielle Stromkosten [€/a]',
            yaxis_title='WP-Model')
fig['data']

/tmp/ipykernel_40872/3091748062.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(Bar({
     'alignmentgroup': 'True',
     'customdata': array([['Bosch Thermotechnik'],
                          ['ait-deutschland'],
                          ['Generic'],
                          ['Master Therm tepelna cerpadla '],
                          ['STIEBEL ELTRON '],
                          ['Vaillant Deutschland '],
                          ['Generic'],
                          ['Viessmann'],
                          ['STIEBEL ELTRON '],
                          ['Nibe'],
                          ['Bosch Thermotechnik']], dtype=object),
     'hovertemplate': ('<b>%{hovertext}</b><br><br>WP-' ... '{customdata[0]}<extra></extra>'),
     'hovertext': array(['CS7800iLW 6 (+F)', 'PWZSV 62H1S', 'Generic Sole/Wasser geregelt',
                         'AquaMaster Inverter AQ17I', 'WPE-I 04 H(K)(W) Premium',
                         'VWF 57/4 35 & 55', 'Generic Sole/Wasser einstufig',
                         'VITOCAL 200-G BWC-M 201.B06', 'WPC 04 cool, average climates

#636efa


In [ ]:
dff = df[(df['Standort'] == standort)&(df['Gebäudetyp']==gebäudetyp)&(df['PV-Ausrichtung']==pv)&(df['WP-Name']==wpname)]
dff['Kosten [1/Jahr]'] = dff['Netzbezug [kWh]'].values * strombezugskosten/100 - dff['Netzeinspeisung [kWh]'].values * einspeisevergütung/100

if wpname=='Generic':
    if wp_name['points'][0]['curveNumber']==0:
        if wp_name['points'][0]['x']=='geregelt':
            dff=dff.loc[(dff['WP-Kategorie']=='Luft/Wasser')&(dff['WP-Typ']=='geregelt')]
        else:
            dff=dff.loc[(dff['WP-Kategorie']=='Luft/Wasser')&(dff['WP-Typ']!='geregelt')]
    else:
        if wp_name['points'][0]['x']=='geregelt':
            dff=dff.loc[(dff['WP-Kategorie']!='Luft/Wasser')&(dff['WP-Typ']=='geregelt')]
        else:
            dff=dff.loc[(dff['WP-Kategorie']!='Luft/Wasser')&(dff['WP-Typ']!='geregelt')]
fig = px.box(dff, x='Batteriespeicher [kWh]',y='Kosten [1/Jahr]', width=1000, height=400)
fig

In [ ]:
# Edit the layout

fig.update_layout(yaxis=dict(range=[dff.sort_values('Kosten [1/Jahr]')['Kosten [1/Jahr]'].head(1),dff.sort_values('Kosten [1/Jahr]')['Kosten [1/Jahr]'].tail(1)]))
